In [37]:
import turicreate as tc
from turicreate import SFrameBuilder
import os
from pathlib import Path
import xml.etree.ElementTree as ET

In [62]:
import ipywidgets as widgets
from IPython.display import display
from decimal import Decimal
out = widgets.Output(layout={'border': '1px solid black'})
twoinplaces = Decimal('0.01')
display(out)

search_dir = '/Volumes/RamDisk/comic'
sb = SFrameBuilder([tc.Image,list],column_names=['image','annotations'])
for root, subdirs, files in os.walk(search_dir):
    for filename in files:
        if filename.endswith(('.png','.jpg','jpeg')):
            with out:
                print("image file",filename)
            file_path = os.path.join(root,filename)
            annfilename=Path(filename).stem + ".annotations"
            annPath = os.path.join(root,annfilename)
            tree = ET.parse(annPath)
            rootElement = tree.getroot()
            print(rootElement.tag)
            annotations=[]
            for subobject in rootElement.findall('object'):
                name=subobject.find('name').text
                bndbox=subobject.find('bndbox')
                xmin=int(bndbox.find('xmin').text)
                xmax=int(bndbox.find('xmax').text)
                ymin=int(bndbox.find('ymin').text)
                ymax=int(bndbox.find('ymax').text)
                bboxDict = {'width': xmax - xmin, 'height': ymax - ymin, 'x':xmin + (xmax - xmin) / 2, 'y':ymin + (ymax - ymin) / 2}
                annotation={'label':name,'coordinates':bboxDict}
                annotations.append(annotation)
            image=tc.Image(file_path)
            sb.append([image,annotations])
sf = sb.close()
sf['image_with_ground_truth'] = \
    tc.object_detector.util.draw_bounding_boxes(sf['image'], sf['annotations'])
sf.explore()

Output(layout=Layout(border='1px solid black'))

annotation
annotation
annotation
annotation


Materializing SFrame

In [ ]:
sf.save(/Volumes/Data/objects.sframe)